In [3]:
import MUR_data_and_machine_learning_model_adapted as mu
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import gc
import random

lat_all = [-4, 32]
lon_all = [44, 90]

step_size = 2 #size of the square, degrees lat by degrees lon
num_epochs = 10 # This is per tile, so don't want this to be too large

lats = np.arange(lat_all[0], lat_all[1], step_size)  
lons = np.arange(lon_all[0], lon_all[1], step_size)



2023-08-10 17:25:50.112298: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-10 17:25:50.114660: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-10 17:25:50.166811: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-10 17:25:50.168248: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-10 17:25:52.114010: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [16]:
#for lat in lats:
#    lat = [lat, lat+step_size]
#    print(lat)


#lat = random.randint(lat_all[0],lat_all[1]-step_size)
#lon = random.randint(lon_all[0],lon_all[1]-step_size)
#print(lat)
#print(lon)


13
73


In [ ]:
#Load the SST data
ds = mu.load_the_dataset()

run_number = 0

#Original idea:
#iterate through different lat/lon squares
#for lat in lats:
#    lat = [lat, lat+step_size]
#    
#    for lon in lons:
#        lon = [lon, lon+step_size]

#New idea:
# Rather than stepping through the lat and lon in 2 loops, randomly select the lat and lon tiles

for num in np.arange(0,10):
    
    #Randomly select lat/lon tiles
    lat = random.randint(lat_all[0],lat_all[1]-step_size)
    lon = random.randint(lon_all[0],lon_all[1]-step_size)
    lat = [lat, lat+step_size]
    lon = [lon, lon+step_size]
    
    #Load the tile:
    print(f'{lat}, {lon}')
    X_train, y_train, X_val, y_val, X_test, y_test = mu.pull_a_tile(ds,lat,lon)

    if run_number == 0:
        #Compile the model:
        model = mu.create_simple_model(np.shape(X_train)[1:] + (1,))
        #model = create_simple_model()
        model.summary()
        model.compile(optimizer='adam', loss='mse', metrics=['mse'])
        early_stop = EarlyStopping(patience=5, restore_best_weights=True)


    #Train the model:
    train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
    train_dataset = train_dataset.shuffle(buffer_size=1024).batch(32)

    val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
    val_dataset = val_dataset.batch(32)

    history = model.fit(train_dataset, epochs=num_epochs, validation_data=val_dataset, callbacks=[early_stop])

    #Increment the run number
    run_number += 1

    #Delete the data to free up memory:
    del X_train, y_train, X_val, y_val, X_test, y_test
    gc.collect()

[17, 19], [83, 85]
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_lstm2d (ConvLSTM2D)    (None, 201, 201, 32)      38144     
                                                                 
 batch_normalization (Batch  (None, 201, 201, 32)      128       
 Normalization)                                                  
                                                                 
 conv2d (Conv2D)             (None, 201, 201, 1)       289       
                                                                 
Total params: 38561 (150.63 KB)
Trainable params: 38497 (150.38 KB)
Non-trainable params: 64 (256.00 Byte)
_________________________________________________________________
Epoch 1/10
1/1 [==============================] - 18s 18s/step - loss: 3.4508 - mse: 3.4508 - val_loss: 2.9642 - val_mse: 2.9642
Epoch 2/10
1/1 [==============================] - 13s 13s/step - 

In [ ]:
explore_results(X_test, model, dscut)